In [1]:
import pandas as pd


    
checkpoints = [
    # "facebook/wav2vec2-large-slavic-voxpopuli-v2",
    # "facebook/wav2vec2-large-960h-lv60-self",
    # "classla/wav2vec2-large-slavic-parlaspeech-hr",
    "facebook/hubert-large-ls960-ft",
]
optimal_epochs = {
    "facebook/wav2vec2-large-960h-lv60-self": 9,
    "facebook/wav2vec2-large-slavic-voxpopuli-v2": 11,
    "classla/wav2vec2-large-slavic-parlaspeech-hr": 7,  
    "facebook/hubert-large-ls960-ft": 10,
}
from utils import train_model, eval_model
for checkpoint in checkpoints:
    import os
    os.system("rm -r models/*")
    train_config = dict(
        model_name_or_path = checkpoint,
        TASK = "emotion_optimal_epochs",
        NUM_EPOCH = optimal_epochs.get(checkpoint),
        output_column = "target",
        input_column = "path",
        data_files = {
            "train": "007_train.csv",
            "validation": "007_dev.csv",
        },
        clip_seconds = 10,
    )

    output_dir = train_model(train_config)
    import numpy as np
    for split in "dev test".split():
        results = []
        from pathlib import Path
        found_path = str(list(Path(output_dir).glob("checkpoint-*"))[0])
        print(f"For checkpoint: {checkpoint} a path was found: ", found_path)
        eval_config = dict(
            output_column = "target",
            model_name_or_path= found_path ,
            eval_file= f"007_{split}.csv"
        )

        y_true, y_pred = eval_model(eval_config)
        import datetime
        time = str(datetime.datetime.now())
        results.append(
            {
                **eval_config, 
                **train_config,
                "split": split,
                "y_true": y_true, 
                "y_pred": y_pred,
                "time": time,
                        })
        content = pd.DataFrame(data=results).to_json(None, 
                                           orient="records",
                                           lines=True,
                                           )
        with open("008_results_new_new_new.jsonl", "a") as f:
            f.writelines(content)

Using custom data configuration default-9260442a7fe0355e
Reusing dataset csv (/home/peterr/.cache/huggingface/datasets/csv/default-9260442a7fe0355e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/2 [00:00<?, ?it/s]

A classification problem with 5 classes: ['anger', 'fear', 'happiness', 'neutral', 'sadness']


Loading cached processed dataset at /home/peterr/.cache/huggingface/datasets/csv/default-9260442a7fe0355e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-daff82a994f3b041.arrow
Loading cached processed dataset at /home/peterr/.cache/huggingface/datasets/csv/default-9260442a7fe0355e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-84ae9b7c5010f384.arrow
Loading cached processed dataset at /home/peterr/.cache/huggingface/datasets/csv/default-9260442a7fe0355e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-a539164cb1745db6.arrow
Loading cached processed dataset at /home/peterr/.cache/huggingface/datasets/csv/default-9260442a7fe0355e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-287f889749556e7d.arrow
Loading cached processed dataset at /home/peterr/.cache/huggingface/datasets/csv/default-9260442a7fe0355e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519

The target sampling rate: 16000


Some weights of the model checkpoint at facebook/hubert-large-ls960-ft were not used when initializing Wav2Vec2ForSpeechClassification: ['hubert.encoder.layers.10.final_layer_norm.bias', 'hubert.encoder.layers.6.attention.q_proj.weight', 'hubert.encoder.layers.9.final_layer_norm.bias', 'hubert.encoder.layers.0.layer_norm.bias', 'hubert.encoder.layers.10.attention.q_proj.weight', 'hubert.encoder.layers.3.feed_forward.intermediate_dense.bias', 'hubert.encoder.layers.12.attention.k_proj.weight', 'hubert.encoder.layers.17.attention.v_proj.bias', 'hubert.encoder.layers.3.attention.v_proj.weight', 'hubert.encoder.layers.22.attention.q_proj.weight', 'hubert.encoder.layers.0.attention.q_proj.bias', 'hubert.encoder.layers.11.feed_forward.output_dense.bias', 'hubert.encoder.layers.11.feed_forward.intermediate_dense.weight', 'hubert.encoder.layers.13.attention.k_proj.weight', 'hubert.encoder.layers.8.attention.v_proj.bias', 'hubert.encoder.layers.7.attention.out_proj.bias', 'hubert.encoder.layers

Epoch,Training Loss,Validation Loss,Accuracy
0,1.714900,1.793728,0.287129
1,1.573500,1.652131,0.178218
2,1.655200,1.626233,0.178218
3,1.795300,1.602993,0.287129
4,1.606300,1.589786,0.287129


The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForSpeechClassification.forward` and have been ignored: valence_annotation_phase, target, split, gender, expression, speaker_id, manual_transcription, arousal_annotation_phase, path, Unnamed: 0, utterance_id, discrete_emotion_collection_phase, age, discrete_emotion_annotation_phase. If valence_annotation_phase, target, split, gender, expression, speaker_id, manual_transcription, arousal_annotation_phase, path, Unnamed: 0, utterance_id, discrete_emotion_collection_phase, age, discrete_emotion_annotation_phase are not expected by `Wav2Vec2ForSpeechClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 202
  Batch size = 2
Saving model checkpoint to models/facebook_hubert-large-ls960-ft_emotion_optimal_epochs_/checkpoint-175
Configuration saved in models/facebook_hubert-large-ls960-ft_emotion_optimal_epochs_/checkpoint-175/config.json
Model wei